In [17]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timezone,timedelta
from nba_api.stats.endpoints import leaguegamefinder
nba_teams = teams.get_teams()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [18]:
elo_df = pd.read_sql_query("SELECT * FROM [2024-2025 Final ELO Rankings]", conn)
elo_dct = {}
for id in elo_df.itertuples():
    elo_dct[id[1]] = id[2]

In [19]:
yesterday = datetime.today() - timedelta(days=1) 
yesterday = yesterday.strftime("%Y-%m-%d")

In [20]:
lst = []
szn_25_26 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2025-10-21",
                                      date_to_nullable=yesterday,
                                      season_type_nullable="Regular Season",
                                      league_id_nullable="00")
szn_25_26 = szn_25_26.get_data_frames()[0]

In [21]:
szn_25_26 = pd.DataFrame(szn_25_26)
ids = list(szn_25_26['TEAM_ID'].unique())
id_to_name  = {team['id']: team['full_name'] for team in nba_teams}

In [22]:
szn_25_26["TotalFGMade"] = szn_25_26.groupby('GAME_ID')['FGM'].transform('sum')
szn_25_26["TotalFGAtt"] = szn_25_26.groupby('GAME_ID')['FGA'].transform('sum')
szn_25_26['Total3PM'] = szn_25_26.groupby('GAME_ID')['FG3M'].transform('sum')
szn_25_26["TotalTOV"] = szn_25_26.groupby('GAME_ID')['TOV'].transform('sum')
szn_25_26["TotalFTA"] = szn_25_26.groupby('GAME_ID')['FTA'].transform('sum')
szn_25_26['TotalOREB'] = szn_25_26.groupby('GAME_ID')['OREB'].transform('sum')
szn_25_26['TotalGamePoints'] = szn_25_26.groupby('GAME_ID')['PTS'].transform('sum')
szn_25_26['OppOREB'] = szn_25_26['TotalOREB'] - szn_25_26['OREB']
szn_25_26['TDREB'] = szn_25_26.groupby('GAME_ID')['DREB'].transform('sum')
szn_25_26['TotalGameFTM'] = szn_25_26.groupby('GAME_ID')['FTM'].transform('sum')
szn_25_26['TotalDREB'] = szn_25_26.groupby('GAME_ID')['DREB'].transform('sum')
szn_25_26['TotalAST'] = szn_25_26.groupby('GAME_ID')['AST'].transform('sum')
szn_25_26['TotalSTL'] = szn_25_26.groupby('GAME_ID')['STL'].transform('sum')
szn_25_26['TotalBLK'] = szn_25_26.groupby('GAME_ID')['BLK'].transform('sum')
szn_25_26['TotalPF'] = szn_25_26.groupby('GAME_ID')['PF'].transform('sum')
szn_25_26['TotalTOV'] = szn_25_26.groupby('GAME_ID')['TOV'].transform('sum')
szn_25_26['GAME_DATE'] = pd.to_datetime(szn_25_26["GAME_DATE"])
copy = szn_25_26.sort_values(by=['TEAM_ID', 'GAME_DATE'])
copy = copy[['GAME_DATE','TEAM_ID', 'GAME_ID', 'TEAM_NAME']]
copy['REST DAYS'] = copy.groupby('TEAM_ID')['GAME_DATE'].diff().dt.days
copy['REST DAYS'] = copy['REST DAYS'] - 1
index = copy.set_index(['GAME_ID', 'TEAM_ID'])
dct = index['REST DAYS'].to_dict()

In [23]:
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 4)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return round(float(x['OREB']) / (float(x['OREB'] + x['OPPDREB'])),4)
def FTRATE(x):
    return round(float(x['FTA']) / float(x['FGA']),4)
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 4)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 4)
def DREBP(x):
    return round(float(x['DREB']) / float(x['DREB'] + x['OppOREB']),4)
def OppFTRATE(x):
    return round(float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA']),4)
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]
def teamPIE(x):
    teampie = (x['PTS'] + x['FGM'] + x['FTM'] - x['FGA'] - x['FTA'] + x['DREB'] + (0.5 * x['OREB']) + x['AST'] + x['STL'] + (x['BLK']*0.5) - x['PF'] - x['TOV']) 
    gamepie = (x['TotalGamePoints'] + x['TotalFGMade'] + x['TotalGameFTM'] - x['TotalFGAtt'] - x['TotalFTA'] + x['TotalDREB'] + (0.5 * x['TotalOREB']) + x['TotalAST'] + x['TotalSTL'] + (0.5 * x['TotalBLK']) - x['TotalPF'] - x['TotalTOV'])
    return round(float(teampie) / float(gamepie), 4)
def away_POSS(x):
    return (x['away_FGA'] + 0.44 * x['away_FTA'] - x['away_OREB'] + x['away_TOV'])
def home_POSS(x):
    return (x['home_FGA'] + 0.44 * x['home_FTA'] - x['home_OREB'] + x['home_TOV'])
def home_PACE(x):
    return (x['home_POSS'] / (x['home_MIN'] / 5)) * 48
def away_PACE(x):
    return (x['away_POSS'] / (x['away_MIN'] / 5)) * 48
def diff(x):
    return abs(x['home_PTS'] - x['away_PTS'])
def elo_diff(x):
    return (x['home_ELO'] - x['away_ELO'])
def pace_DIFF(x):
    return (x['home_PACE'] - x['away_PACE'])
def rest_DIFF(x):
    return (x['home REST DAYS'] - x['away REST DAYS'])
def home_offRating(x):
    return ((x['home_PTS'] * 100) / x['home_POSS'])
def away_offRating(x):
    return ((x['away_PTS'] * 100) / x['away_POSS'])
def home_defRating(x):
    return ((x['away_PTS'] * 100) / x['away_POSS'])
def away_defRating(x):
    return ((x['home_PTS'] * 100) / x['home_POSS'])

In [24]:
szn_25_26['EFG'] = szn_25_26.apply(EFG, axis=1)
szn_25_26['TOVP'] = szn_25_26.apply(TOVP, axis=1)
szn_25_26['OREBP'] = szn_25_26.apply(OREBP, axis=1)
szn_25_26['FTRATE'] = szn_25_26.apply(FTRATE, axis=1)
szn_25_26['OppEFG'] = szn_25_26.apply(OppEFG, axis=1)
szn_25_26['OppTOVP'] = szn_25_26.apply(OppTOVP, axis=1)
szn_25_26['DREBP'] = szn_25_26.apply(DREBP, axis=1)
szn_25_26['OppFTRATE'] = szn_25_26.apply(OppFTRATE, axis=1)
szn_25_26["AWAY"] = szn_25_26.apply(away, axis=1)
szn_25_26["HOME"] = szn_25_26.apply(home, axis=1)
szn_25_26['TEAMPIE'] = szn_25_26.apply(teamPIE, axis=1)

In [25]:
szn_25_26.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)

In [26]:
def combine_game_rows(df):
    df = df.copy()
    df['is_home'] = df['MATCHUP'].str.contains('vs.', na=False)
    stat_columns = [col for col in df.columns if col not in 
                   ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 
                    'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'AWAY', 'HOME']]
    home_data = df[df['is_home'] == True].copy()
    away_data = df[df['is_home'] == False].copy()
    home_rename = {col: f'home_{col}' for col in stat_columns}
    home_rename.update({
        'TEAM_ID': 'home_TEAM_ID',
        'TEAM_ABBREVIATION': 'home_TEAM',
        'TEAM_NAME': 'home_TEAM_NAME',
        'WL': 'home_WL'
    })
    home_data = home_data.rename(columns=home_rename)
    away_rename = {col: f'away_{col}' for col in stat_columns}
    away_rename.update({
        'TEAM_ID': 'away_TEAM_ID',
        'TEAM_ABBREVIATION': 'away_TEAM',
        'TEAM_NAME': 'away_TEAM_NAME',
        'WL': 'away_WL'
    })
    away_data = away_data.rename(columns=away_rename)
    merge_cols = ['GAME_ID', 'GAME_DATE']
    if 'SEASON_ID' in df.columns:
        merge_cols.append('SEASON_ID')
    combined = pd.merge(
        home_data[merge_cols + [col for col in home_data.columns if col not in merge_cols]],
        away_data[merge_cols + [col for col in away_data.columns if col not in merge_cols]],
        on=merge_cols,
        how='inner'
    )
    home_ewma_cols = [col for col in combined.columns if 'home_EWMA' in col]
    away_ewma_cols = [col for col in combined.columns if 'away_EWMA' in col]
    for x,y in zip(home_ewma_cols, away_ewma_cols):
        name = x.split(" ")
        combined[f'EWMA {name[-1]} DIFF'] = combined[f'{x}'] - combined[f'{y}']
    combined['home_win'] = (combined['home_WL'] == 'W').astype(int)
    return combined

In [27]:
def ELO(x):
    prob_home = 1 / (1 + 10**((elo_dct[x['away_TEAM_ID']] - elo_dct[x['home_TEAM_ID']] - 100)/400))
    prob_away = 1 - prob_home
    if (x['home_win'] == 1):
        pre_elo_home = elo_dct[x['home_TEAM_ID']]
        elo_dct[x['home_TEAM_ID']] = pre_elo_home + 20 * (1 - prob_home)
        pre_elo_away = elo_dct[x['away_TEAM_ID']]
        elo_dct[x['away_TEAM_ID']] = pre_elo_away + 20 * (0 - prob_away)
    else:
        pre_elo_home = elo_dct[x['home_TEAM_ID']]
        elo_dct[x['home_TEAM_ID']] = pre_elo_home + 20 * (0 - prob_home)
        pre_elo_away = elo_dct[x['away_TEAM_ID']]
        elo_dct[x['away_TEAM_ID']] = pre_elo_away + 20 * (1 - prob_away)
    return pre_elo_home, pre_elo_away

In [28]:
szn_25_26['_original_index'] = range(len(szn_25_26))
x_sorted = szn_25_26.sort_values(['TEAM_ID', 'SEASON_ID', 'GAME_DATE']).reset_index(drop=True)
ewma_cols = ['PTS', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
'OREB', 'DREB', 'AST', 'STL', 'TOV', 'PF', 'PLUS_MINUS', 'EFG', 'TOVP',
'OREBP', 'FTRATE', 'OppEFG', 'OppTOVP', 'DREBP', 'OppFTRATE', 'TEAMPIE']
for col in ewma_cols:
    x_sorted[f"EWMA {col}"] = (
    x_sorted.groupby(['TEAM_ID', 'SEASON_ID'])[col].apply(lambda x: x.shift(1).ewm(span=7, min_periods=7).mean().round(4)).reset_index(level=[0, 1], drop=True))
x_sorted = x_sorted.sort_values('_original_index').reset_index(drop=True)
x_sorted = x_sorted.drop(columns=['_original_index'])

In [29]:
combined_df = combine_game_rows(x_sorted)
combined_df['DIFF'] = combined_df.apply(diff, axis=1)
combined_df.drop(columns=['AWAY_y','HOME_y', 'away_is_home','home_is_home','AWAY_x', 'HOME_x',], inplace=True)
combined_df = combined_df.sort_values(by='GAME_DATE', ascending=True)
combined_df[['home_ELO', 'away_ELO']] = combined_df.apply(ELO, axis=1, result_type='expand')
home_ref = pd.Series(zip(combined_df['GAME_ID'], combined_df['home_TEAM_ID']),index=combined_df.index)
away_ref = pd.Series(zip(combined_df['GAME_ID'] , combined_df['away_TEAM_ID']),index=combined_df.index)
combined_df['home REST DAYS'] = pd.Series(home_ref).map(dct)
combined_df['away REST DAYS'] = pd.Series(away_ref).map(dct)
combined_df['away_POSS'] = combined_df.apply(away_POSS,axis=1)
combined_df['home_POSS'] = combined_df.apply(home_POSS,axis=1)
combined_df['away_PACE'] = combined_df.apply(away_PACE, axis=1)
combined_df['home_PACE'] = combined_df.apply(home_PACE, axis=1)
combined_df['elo_DIFF'] = combined_df.apply(elo_diff, axis=1)
combined_df['rest DIFF'] = combined_df.apply(rest_DIFF, axis=1)
combined_df['home_OffRating'] = combined_df.apply(home_offRating, axis=1)
combined_df['away_OffRating'] = combined_df.apply(away_offRating, axis=1)
combined_df['home_DefRating'] = combined_df.apply(home_defRating, axis=1)
combined_df['away_DefRating'] = combined_df.apply(away_defRating, axis=1)

In [30]:
print(len(list(combined_df.columns)))

176


In [31]:
combined_df

,GAME_ID,GAME_DATE,SEASON_ID,home_TEAM_ID,home_TEAM,home_TEAM_NAME,MATCHUP_x,home_WL,home_MIN,home_PTS,home_FGM,home_FGA,home_FG_PCT,home_FG3M,home_FG3A,home_FG3_PCT,home_FTM,home_FTA,home_FT_PCT,home_OREB,home_DREB,home_REB,home_AST,home_STL,home_BLK,home_TOV,home_PF,home_PLUS_MINUS,home_TotalGamePoints,home_TotalGameFTM,home_TotalDREB,home_TotalAST,home_TotalSTL,home_TotalBLK,home_TotalPF,home_EFG,home_TOVP,home_OREBP,home_FTRATE,home_OppEFG,home_OppTOVP,home_DREBP,home_OppFTRATE,home_TEAMPIE,home_EWMA PTS,home_EWMA FGA,home_EWMA FG_PCT,home_EWMA FG3M,home_EWMA FG3A,home_EWMA FG3_PCT,home_EWMA FTM,home_EWMA FTA,home_EWMA FT_PCT,home_EWMA OREB,home_EWMA DREB,home_EWMA AST,home_EWMA STL,home_EWMA TOV,home_EWMA PF,home_EWMA PLUS_MINUS,home_EWMA EFG,home_EWMA TOVP,home_EWMA OREBP,home_EWMA FTRATE,home_EWMA OppEFG,home_EWMA OppTOVP,home_EWMA DREBP,home_EWMA OppFTRATE,home_EWMA TEAMPIE,away_TEAM_ID,away_TEAM,away_TEAM_NAME,MATCHUP_y,away_WL,away_MIN,away_PTS,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FTM,away_FTA,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_STL,away_BLK,away_TOV,away_PF,away_PLUS_MINUS,away_TotalGamePoints,away_TotalGameFTM,away_TotalDREB,away_TotalAST,away_TotalSTL,away_TotalBLK,away_TotalPF,away_EFG,away_TOVP,away_OREBP,away_FTRATE,away_OppEFG,away_OppTOVP,away_DREBP,away_OppFTRATE,away_TEAMPIE,away_EWMA PTS,away_EWMA FGA,away_EWMA FG_PCT,away_EWMA FG3M,away_EWMA FG3A,away_EWMA FG3_PCT,away_EWMA FTM,away_EWMA FTA,away_EWMA FT_PCT,away_EWMA OREB,away_EWMA DREB,away_EWMA AST,away_EWMA STL,away_EWMA TOV,away_EWMA PF,away_EWMA PLUS_MINUS,away_EWMA EFG,away_EWMA TOVP,away_EWMA OREBP,away_EWMA FTRATE,away_EWMA OppEFG,away_EWMA OppTOVP,away_EWMA DREBP,away_EWMA OppFTRATE,away_EWMA TEAMPIE,EWMA PTS DIFF,EWMA FGA DIFF,EWMA FG_PCT DIFF,EWMA FG3M DIFF,EWMA FG3A DIFF,EWMA FG3_PCT DIFF,EWMA FTM DIFF,EWMA FTA DIFF,EWMA FT_PCT DIFF,EWMA OREB DIFF,EWMA DREB DIFF,EWMA AST DIFF,EWMA STL DIFF,EWMA TOV DIFF,EWMA PF DIFF,EWMA PLUS_MINUS DIFF,EWMA EFG DIFF,EWMA TOVP DIFF,EWMA OREBP DIFF,EWMA FTRATE DIFF,EWMA OppEFG DIFF,EWMA OppTOVP DIFF,EWMA DREBP DIFF,EWMA OppFTRATE DIFF,EWMA TEAMPIE DIFF,home_win,DIFF,home_ELO,away_ELO,home REST DAYS,away REST DAYS,away_POSS,home_POSS,away_PACE,home_PACE,elo_DIFF,rest DIFF,home_OffRating,away_OffRating,home_DefRating,away_DefRating
175,0022500002,2025-10-21,22025,1610612747,LAL,Los Angeles Lakers,LAL vs. GSW,L,240,109,42,77,0.545,8,32,0.250,17,28,0.607,7,32,39,23,7,2,19,21,-10.0,228,43,63,52,17,6,48,0.5974,0.1754,0.1842,0.3636,0.5962,0.1655,0.7805,0.3718,0.4543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612744,GSW,Golden State Warriors,GSW @ LAL,W,241,119,38,78,0.487,17,40,0.425,26,29,0.897,9,31,40,29,10,4,18,27,10.0,228,43,63,52,17,6,48,0.5962,0.1655,0.2195,0.3718,0.5974,0.1754,0.8158,0.3636,0.5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,1564.153644,1560.154022,NaN,NaN,99.76,101.32,99.346058,101.320000,3.999622,NaN,107.579945,119.286287,119.286287,107.579945
174,0022500001,2025-10-21,22025,1610612760,OKC,Oklahoma City Thunder,OKC vs. HOU,W,290,125,46,104,0.442,13,52,0.250,20,25,0.800,11,27,38,29,12,4,11,27,1.0,249,47,63,52,18,9,53,0.5048,0.0873,0.2340,0.2404,0.5000,0.1659,0.6279,0.3196,0.5155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612745,HOU,Houston Rockets,HOU @ OKC,L,292,124,43,97,0.443,11,39,0.282,27,31,0.871,16,36,52,23,6,5,22,26,-1.0,249,47,63,52,18,9,53,0.5000,0.1659,0.3721,0.3196,0.5048,0.0873,0.7660,0.2404,0.4845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1672.840582,1560.714882,NaN,NaN,116.64,115.00,95.868493,

In [32]:
combined_df.to_sql("2025-2026 Team Data Combined", conn, if_exists="replace", index=False)

176